In [ ]:
# import torch
# print(f"\n🔍 PyTorch: {torch.__version__}")
# print(f"🔍 GPU: {torch.cuda.is_available()}")
# print(f"🔍 Nome da GPU: {torch.cuda.get_device_name(0)}")

# # Nova forma de verificar ROCm a partir do PyTorch 2.0+
# print(f"🔍 Backend GPU: {'ROCm' if torch.version.hip else 'CUDA'}")
# print(f"🔍 Versão HIP: {torch.version.hip}\n")

# Carregando Pacotes e Tokenizador:

In [ ]:
pip install pdfplumber

In [ ]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer



In [ ]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

# Variáveis Globais

In [ ]:
#Tamanho da Janela de Contexto
window_size=400

#Overlap da Janela de Contexto
stride=100

#Tamanho da Chunk
chunk_size = 400

#Overlap da Chunk
overlap = 100

#Tamanho Máximo da Pergunta
tamanho_pergunta = 60

In [ ]:
modelo_tokenizador = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

# Tratamento do Documento

In [ ]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained(modelo_tokenizador)

qa_pipeline = pipeline(
    "question-answering",
    model= modelo_tokenizador,  # Modelo em Portugues
    tokenizer=modelo_tokenizador,
    device=device
)

In [ ]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages
            if page.extract_text()
        )
        texto = " ".join(texto.split())
    return texto

## Divisão em Chunks

In [ ]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

## Janela Deslizante

In [ ]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []

    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)

        if i + window_size >= len(tokens):
            break

    return janelas

# Definição de Processamento das Perguntas e das Respostas

In [ ]:
def processar_pergunta(pergunta, documento):

    chunks = divisao_do_texto

    # 1. Tokeniza a pergunta para verificar tamanho
    tokens_pergunta = tokenizer.tokenize(pergunta)
    if len(tokens_pergunta) > tamanho_pergunta:  # Limite arbitrário (ajuste conforme necessário)
        print("Pergunta muito longa! Simplifique para melhor precisão.")

    # 2. Executa Q&A em cada chunk
    respostas = []
    for chunk in chunks:
        try:
            resposta = qa_pipeline(question=pergunta, context=chunk)
            respostas.append(resposta)

            resposta_completa = {
                'answer': resposta.get('answer', ''),
                'score': resposta.get('score', 0),
                'context': resposta.get('context', chunk[:500])  # Fallback: 500 primeiros chars
            }
            respostas.append(resposta_completa)
        except Exception as e:
            print(f"Erro no chunk: {str(e)}")
            continue

    # 3. Filtra respostas com score baixo e seleciona a melhor
    respostas_validas = [r for r in respostas if r['score'] >= 0.2]
    if not respostas_validas:
        print("Não há resposta sobre isto nesse documento.")
        return None

    return max(respostas_validas, key=lambda x: x['score'])

# Pergunta e Resposta

In [ ]:
#Carrega o documento em PDF

documento = carregar_documento("Cabelo Cacheado Maior.pdf")

In [ ]:
#Divisao em Janela Deslizante

#divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

divisao_do_texto = dividir_em_chunks_tokenizados(documento)

In [ ]:
#Pergunta sobre o documento
pergunta = "Como é a estrutura do cacho tipo 4?"

In [ ]:
#Resposta gerada
resposta = processar_pergunta(pergunta, documento)
print(resposta)